In [ ]:
%pip install brian2
%pip install brian2tools
%pip install scipy
!pip install --upgrade scipy

In [ ]:
from brian2 import *
from brian2tools import *
import numpy as np
import pandas as pd
from scipy import io
from scipy import sparse
from google.colab import drive
drive.mount('/content/gdrive')
import sys
sys.path.append('/content/gdrive/MyDrive/Figures_Package')
import seaborn as sns

Mounted at /content/gdrive


In [ ]:
#this function takes as input the first and second momentum of the weights population given in supp table 1 
#it transforms the average and var into the average and std of a lognormal distribution (formulas taken from wikipedia)

def process_params(av, av2):
    var = (av2 - av**2)
    
    mu = np.log(av**2/(av**2 + var)**(1/2))
    sigma = np.log(1 + var/av**2)
    
    return mu, sigma

In [ ]:
#the weight distribution parameters are processed and stored in non-array variables to be passed to brian
#in the future let's just do this computation and store the values in a file to read

avs = np.array([0.37, 0.66, 0.44, 0.54])
avs2 = np.array([0.26, 0.65, 0.49, 0.53])

lognormal_mu = []
lognormal_std = []

for i in range(len(avs)):
    mu, sigma = process_params(avs[i], avs2[i])
    lognormal_mu.append(mu)
    lognormal_std.append(sigma)
    
ee_mu = lognormal_mu[0]
ee_std = lognormal_std[0]
ie_mu = lognormal_mu[1]
ie_std = lognormal_std[1]
ei_mu = lognormal_mu[2]
ei_std = lognormal_std[2]
ii_mu = lognormal_mu[3]
ii_std = lognormal_std[3]

In [ ]:
initial_v = np.load('/content/gdrive/MyDrive/Figures_Package/Networks/initial_voltage.npy')

In [ ]:
#define network parameters

#total size of the network
N = 40000

#threshold, reset and refractory parameters
v_theta = 33*mV
v_reset = 24.75*mV
tau_ref = 1*ms
v_0 = -1000*mV

#indices that delimit inhibitory and excitatory populations 20/80
first_inh = 0
last_inh = int(0.2*N)
first_exc = last_inh
last_exc = N


#these are additional parameters to play with resizing of weights (in paper would be 1)
g_E = 1
g_I = 1


#External drive to each population

H_in = 57.8*mV
H_ex = 77.6*mV

tau_in = np.log((H_in - v_reset)/(H_in - v_theta))*second;
tau_ex = np.log((H_ex - v_reset)/(H_ex - v_theta))*second;


#clear brian scope to reset any past variables
start_scope()

#define the model that each neuron will follow
tau = 10*ms
eqs = '''
dv/dt = -(v - H_ext)/tau : volt (unless refractory)
H_ext : volt
'''

#the original network is defined
all_neurons = NeuronGroup(N, eqs, threshold = 'v>v_theta', reset = 'v=v_reset', refractory = tau_ref, method = 'exact')

#to work with subpopulations bria-n uses slicing notation
in_neurons = all_neurons[first_inh:last_inh]
ex_neurons = all_neurons[first_exc:last_exc]
targ_neurons = ex_neurons[:6400]

all_neurons.v = initial_v*mV


#define external drive according to supp table 1
in_neurons.H_ext = H_in
ex_neurons.H_ext = H_ex
#in_neurons.H = '(24 + 9.1*rand())*mV'
#ex_neurons.H = '(24 + 9.1*rand())*mV'


#initialize the synaptic object for the different subpopulation pairs
S_EE = Synapses(ex_neurons, ex_neurons, 'w : volt', on_pre ='v += w')
S_IE = Synapses(ex_neurons, in_neurons, 'w : volt', on_pre ='v += w')
S_EI = Synapses(in_neurons, ex_neurons,'w : volt', on_pre ='v -= w')
S_II = Synapses(in_neurons, in_neurons, 'w : volt', on_pre ='v -= w')


#connect them according to probabilities in supp table 1
S_EE.connect(p = 0.2)
S_IE.connect(p = 0.3)
S_EI.connect(p = 0.4)
S_II.connect(p = 0.4)



#distribute them log-normal according to parameters obtained in processing cells (beginning of notebook)
S_EE.w = 'g_E*exp(ee_mu + ee_std*randn())*mV'
S_IE.w = 'g_E*exp(ie_mu + ie_std*randn())*mV'
S_EI.w = 'g_I*exp(ei_mu + ei_std*randn())*mV'
S_II.w = 'g_I*exp(ii_mu + ii_std*randn())*mV'



#define monitoring variables. these can monitor the whole population or a desired subpopulation

#monitors spiking events
M_in = SpikeMonitor(in_neurons)
M_ex = SpikeMonitor(ex_neurons)

#V_in = StateMonitor(in_neurons, 'v', record = [1, 100, 1000])
#V_ex = StateMonitor(ex_neurons, 'v', record = [1, 100, 1000])

#monitors voltage with time
#V = StateMonitor(all_neurons, 'v', record = [1, 100, 10000, 32000])

In [ ]:
defaultclock.dt = 0.005*ms
total_time = 1000000*ms

run(total_time, report = 'text')

In [ ]:
def process_firings(M_in, M_ex, time):
    global session
    count_in = np.array(M_in.count)/(time/second)
    count_ex = np.array(M_ex.count)/(time/second)
    count_in_path = 'FR_in_session' + str(session) + '.npy'
    count_ex_path = 'FR_ex_session' + str(session) + '.npy'
    np.save(count_in_path, count_in)
    np.save(count_ex_path, count_ex)




In [ ]:
time=total_time
session = 1
process_firings(M_in, M_ex, time)

In [ ]:
FR_in = np.load('FR_in_session1.npy')

In [ ]:
plt.hist(FR_in, bins = 'auto')
plt.title("histogram")
plt.show()

In [ ]:
sns.distplot(FR_in, hist=True, kde=True,
bins=int(180/5), color = 'red',
hist_kws={'edgecolor':'black'},
kde_kws={'linewidth': 2})
plt.xlim(xmin=0, xmax = 30)
plt.xlabel('Firing Rate (Hz)')
plt.ylabel('Probability Density ($Hz^-1$)')

In [ ]:
FR_ex = np.load('FR_ex_session1.npy')

In [ ]:
plt.hist(FR_ex, bins = 'auto')
plt.title("histogram")
plt.show()

In [ ]:
sns.distplot(FR_ex, hist=True, kde=True,
bins=int(180/5), color = 'blue',
hist_kws={'edgecolor':'black'},
kde_kws={'linewidth': 2})
plt.xlim(xmin=0, xmax = 20)
plt.xlabel('Firing Rate (Hz)')
plt.ylabel('Probability Density ($Hz^-1$)')

In [ ]:
#pdf = norm.pdf(FR_ex , loc = 1.01 , scale = 1.01) 
sns.distplot(FR_in, hist=True, kde=True,
             bins=np.logspace(np.log10(0.1), np.log10(100), 25), color = 'red', 
             hist_kws={'edgecolor':'black'},
             kde_kws={'linewidth': 2})
plt.xscale('log')

In [ ]:
sns.distplot(FR_ex, hist=True, kde=True,
             bins=np.logspace(np.log10(0.01), np.log10(10), 25), color = 'darkblue', 
             hist_kws={'edgecolor':'black'},
             kde_kws={'linewidth': 2})
plt.xscale('log')